In [1]:
from path import setup_project_root

setup_project_root()

In [2]:
import glob 

base = '/om2/user/moshepol/prosody/models/n_fft=1024_hop_length=256/'

inputs = glob.glob(f'{base}inputs*.pt')
labels = glob.glob(f'{base}outputs*.pt')

In [3]:
print(f'Total Inputs: {len(inputs)}')
print(f'Total Outputs: {len(labels)}')

Total Inputs: 14
Total Outputs: 14


In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset

tensors = []
first = {'train' : {'en' : ''}}

for i, (inp, out) in enumerate(zip(inputs, labels)):

    if i % 5 == 0:
        print(inp.split('/')[-1])
        print(out.split('/')[-1])

    if i == 0:
        first['train']['en'] = torch.load(inp, weights_only=False)

    tensors.append(
        TensorDataset(
            torch.load(inp, weights_only=False), 
            torch.load(out, weights_only=False)
            )
        )

inputs_0.pt
outputs_0.pt


In [5]:
data = ConcatDataset(tensors)
loader = DataLoader(data, batch_size=256, num_workers=4, shuffle=False)

In [7]:
first['train']['en'].shape

torch.Size([256, 513, 344])

In [8]:
product = 1
for val in first['train']['en'][0].shape:
    if val % 2 == 0:
        product *= val
    else:
        product *= (val - 1)

In [9]:
from Multi_Lang_GPU import LanguageDetector, calc_middle_neuron

num_classes = 4
middle_neuron = product

model = LanguageDetector(num_classes, middle_neuron)

base = '/om2/user/moshepol/prosody/models/n_fft=1024_hop_length=256/best_model.pth'
model.load_state_dict(torch.load(base, weights_only=True))
model.eval()

model.to('cuda')

LanguageDetector(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=352256, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [10]:
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, label in loader:
        print(inputs.shape)

        inputs = inputs.to('cuda')
        label = label.to('cuda')

        inputs = inputs.unsqueeze(1)

        outputs = model(inputs)

        _, predicted = torch.max(outputs, dim=1)

        y_pred.append(predicted)
        y_true.append(label)

torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])
torch.Size([256, 513, 344])


In [11]:
y_pred = torch.cat(y_pred).cpu()
y_true = torch.cat(y_true).cpu()

In [12]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))

Accuracy: 0.6623883928571429

Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.63      0.63       896
           1       0.63      0.65      0.64       896
           2       0.63      0.66      0.64       896
           3       0.75      0.72      0.74       896

    accuracy                           0.66      3584
   macro avg       0.66      0.66      0.66      3584
weighted avg       0.66      0.66      0.66      3584


Confusion Matrix:
 [[561 179  96  60]
 [154 582 117  43]
 [ 89 114 587 106]
 [ 83  43 126 644]]
